In [48]:
import pandas as pd
from selenium import webdriver
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import collections
from selenium.common.exceptions import TimeoutException
import pandas as pd
import requests
import time

## Degree Scrape

In [49]:

# Read data from Excel file
degree = input("Enter degree: ")
# df = pd.DataFrame({"Column 1": ["Master of Data Science", "Bachelor of Business"]})
# Create a new column with modified titles
df = pd.DataFrame({"Column 1": [degree]})
df["Modified Title"] = degree.replace(" ", "-").lower()

firefox_option = Options()
firefox_option.add_argument("--headless")
driver = webdriver.Firefox(options=firefox_option)

# Click cancel button
cancel = True

not_found = []
all_degree = collections.defaultdict(lambda: collections.defaultdict(dict))

# Loop through the DataFrame rows
for index, row in df.iterrows():
    # Get the modified title and construct the URL
    modified_title = row["Modified Title"]
    print("Modified title: ", modified_title)
    # Construct the URL with the modified title
    url = f"https://www.mq.edu.au/study/find-a-course/courses/2024/{modified_title}#course-structure"
    print("URL: ", url)

    # Open the webpage
    driver.get(url)

    if cancel:
        time.sleep(1)
        # Find the button element
        button = driver.find_element(
            By.XPATH, "/html/body/div[1]/div[1]/div[2]/aside[2]/div/div/button"
        )
        button.location_once_scrolled_into_view
        time.sleep(2)
        # Click the button
        button.click()
        cancel = False

    # Edge case: 2024
    # Search for the "Page Not Found" heading
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    page_not_found_heading = soup.find("h1", text="Page Not Found")
    if page_not_found_heading:
        try:
            if page_not_found_heading:
                url = f"https://www.mq.edu.au/study/find-a-course/courses/{modified_title}#course-structure"
                # Open the alternative URL
                driver.get(url)
                time.sleep(0.4)
                html = driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                wait = WebDriverWait(driver, 1)  # Maximum wait time of 10 seconds
                wait.until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "tbody.padded.table--expandable")
                    )
                )

        except TimeoutException:
            not_found.append(row["Column 1"])
            continue
    try:
        wait = WebDriverWait(driver, 1)  # Maximum wait time of 10 seconds
        wait.until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "tbody.padded.table--expandable")
            )
        )

    except TimeoutException:
        # Check if the heading is found
        not_found.append(row["Column 1"])
        continue

    print("Not found: ", not_found)

    ## SCRAPE
    # all_degree["Bachelor of Business"]
    # Find all <div> elements with class "flex order-md--1 margin--16"
    time.sleep(0.1)
    divs = soup.find_all("div", class_="table--structure-subhead")

    course_type = []
    # Iterate over the found divs
    for div in divs:
        # Find the first <div> element inside the current div
        inner_div = div.find("h5")

        # Extract the text content of the inner <div>
        course = inner_div.text

        # Print the extracted text
        course_type.append(course)
    print("Course type: ", course_type)
    time.sleep(0.1)
    # Find all <div> elements with class "flex order-md--1 margin--16"
    divs = soup.find_all("tbody", class_="padded table--expandable")
    count = 0
    for div in divs:
        if len(course_type) == 0:
            not_found.append(row["Column 1"])
            break
        inner_divs = div.find_all(["p", "a"])
        courses = []
        for p in inner_divs:
            # Extract the text content of the inner <div>
            course = p.text
            # if "flexible zone" in course.lower():
            #     break
            print(course)
            courses.append(course)
        # if count >= len(course_type):
        #     not_found.append(row["Column 1"])
        #     break
        # print("row: ", row["Column 1"])
        # print("div: ", divs)
        # print(soup)
        all_degree[row["Column 1"]][course_type[count]] = (
            list(all_degree[row["Column 1"]][course_type[count]]) + courses
        )
        count += 1
        if count >= len(course_type):
            break
    print("-------------------")

## EXPORT TO EXCEL
data = []
for degree, categories in all_degree.items():
    for category, courses in categories.items():
        data.extend(
            [(degree, category, *courses[i : i + 2]) for i in range(0, len(courses), 2)]
        )

df_degree = pd.DataFrame(data, columns=["Degree", "Category", "Course Code", "Course Name"])

Modified title:  bachelor-of-planning
URL:  https://www.mq.edu.au/study/find-a-course/courses/2024/bachelor-of-planning#course-structure
Not found:  []
Course type:  ['Essential units', 'Capstone unit', 'Elective units']
ABST1020
Dharug Ngurra
ENVS1017
The Living Environment
ENVS1018
Environmental Management for a Changing World
GEOP1010
Introducing Human Geography
GEOP1030
Sydney and the Great Australian Dream
GEOP1040
Planning Systems
GEOP1080
Urban Planet: Cities and Planning in the Anthropocene
ENVS2364
Introduction to Geographic Information Science and Remote Sensing
GEOP2010
Theory and Methods in Geography and Planning
GEOP2040
Cities of the Global South 
GEOP2050
Environment and Society
GEOP2060
Geography and Planning Field School
GEOP2070
Urban Dynamics: Population, Housing and Regeneration
GEOP2080
Planning and Development
GEOP3000
Environmental Justice in the Anthropocene
GEOP3020
Disasters and Resilient Communities 
GEOP3030
Liveable Cities: Infrastructure, Mobilities and Pl

In [50]:
df_degree

,Degree,Category,Course Code,Course Name
0,Bachelor of Planning,Essential units,ABST1020,Dharug Ngurra
1,Bachelor of Planning,Essential units,ENVS1017,The Living Environment
2,Bachelor of Planning,Essential units,ENVS1018,Environmental Management for a Changing World
3,Bachelor of Planning,Essential units,GEOP1010,Introducing Human Geography
4,Bachelor of Planning,Essential units,GEOP1030,Sydney and the Great Australian Dream
5,Bachelor of Planning,Essential units,GEOP1040,Planning Systems
6,Bachelor of Planning,Essential units,GEOP1080,Urban Planet: Cities and Planning in the Anthr...
7,Bachelor of Planning,Essential units,ENVS2364,Introduction to Geographic Information Science...
8,Bachelor of Planning,Essential units,GEOP2010,Theory and Methods in Geography and Planning
9,Bachelor of Planning,Essential units,GEOP2040,Cities of the Global South


In [51]:
updated_df

,Degree,Category,Course Code,Course Name,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Bachelor of Actuarial Studies,Essential units,ACST1052,Introduction to Actuarial Studies,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bachelor of Actuarial Studies,Essential units,ECON1020,Principles of Economics 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bachelor of Actuarial Studies,Essential units,ECON1021,Principles of Economics 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bachelor of Actuarial Studies,Essential units,MATH1025,Calculus and Linear Algebra II (Advanced),NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bachelor of Actuarial Studies,Essential units,STAT1371,Statistical Data Analysis,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2420,Bachelor of Business Administration,Capstone unit,MGMT3060,PACE: Business Project,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2421,Bachelor of Business Administration,Elective units,ACST1001,Finance 1A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2422,Bachelor of Business Administration,Elective units,MGMT1007,Introduction to Human Resource Management,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2423,Bachelor of Business Administration,Elective units,MGMT1020,Entrepreneurship and New Venture Management,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
title_all = "course_data3"
# Existing DataFrame
existing_df = pd.read_excel(f"{title_all}.xlsx")
# Append the new data to the existing DataFrame
updated_df = existing_df.append(df_degree, ignore_index=True)
# Drop duplicates from the combined DataFrame
updated_df = updated_df.drop_duplicates()
# Export the DataFrame to an Excel file
filename = f"{title_all}.xlsx"
sheet_name = f"{title_all}"
updated_df.to_excel(filename, sheet_name=sheet_name, index=False)

C:\Users\nplon\AppData\Local\Temp\ipykernel_85016\2225042340.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updated_df = existing_df.append(df_degree, ignore_index=True)


## Handbook

In [53]:
# Remove duplicate course codes and get corresponding course names
unique_courses = df_degree.drop_duplicates(subset="Course Code")
course_codes = unique_courses["Course Code"]
course_names = unique_courses["Course Name"]

all_course_code = []
all_course_name = []
period = []
error_list = []
firefox_option = Options()
firefox_option.add_argument("--headless")
driver = webdriver.Firefox(options=firefox_option)
# Scrape the website for each course code
for course_code, course_name in zip(course_codes, course_names):
    url = f"https://coursehandbook.mq.edu.au/2023/units/{course_code}"
    driver.get(url)
    time.sleep(0.3)
    html = driver.page_source
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # Find the div with the ID "Offerings"
    div_offerings = soup.find("div", id="Offerings")

    if div_offerings:
        # Find all div elements within the div_offerings element
        div_elements = div_offerings.find_all("div")
        print(len(div_elements))

        if len(div_elements) >= 2:
            # Get the second div element
            second_div = div_elements[3]
            sessions = second_div.get_text(strip=True).split("Session ")
            formatted_sessions = "\n".join(
                ["Session " + s for s in sessions if s.strip()]
            )
            print(formatted_sessions)
            period.append(formatted_sessions)
            all_course_code.append(course_code)
            all_course_name.append(course_name)

    # # Process the response as needed
    # # For example, you can print the course code, course name, and response status code
    print(f"Course Code: {course_code}")
    print(f"Course Name: {course_name}")
    print("------------------------------")

## EXPORT TO EXCEL FILE
# Assuming you have the following lists:
# Create a DataFrame with the data
data = {
    "Course Code": all_course_code,
    "Course Name": all_course_name,
    "Period": period,
}
df = pd.DataFrame(data)

4
Session 1, In person-scheduled-weekday, North Ryde
Session 1, Online-flexible
Session 1, Online-scheduled-weekday
Session 2, In person-scheduled-weekday, North Ryde
Session 2, Online-flexible
Session 2, Online-scheduled-weekday
Course Code: ABST1020
Course Name: Dharug Ngurra
------------------------------
4
Session 1, In person-scheduled-weekday, North Ryde
Session 1, In person-scheduled-infrequent, North Ryde
Course Code: ENVS1017
Course Name: The Living Environment
------------------------------
4
Session 2, In person-scheduled-weekday, North Ryde
Session 2, Online-flexible, Other
Course Code: ENVS1018
Course Name: Environmental Management for a Changing World
------------------------------
4
Session 1, In person-scheduled-weekday, North Ryde
Session 1, Online-flexible
Course Code: GEOP1010
Course Name: Introducing Human Geography
------------------------------
4
Session 2, In person-scheduled-weekday, North Ryde
Course Code: GEOP1030
Course Name: Sydney and the Great Australian D

## StudentVIP

In [54]:
reviews = []
stars = []
not_found = []
unique_courses = df.drop_duplicates(subset="Course Code")
course_codes = unique_courses["Course Code"]
course_names = unique_courses["Course Name"]
firefox_option = Options()
firefox_option.add_argument("--headless")
driver = webdriver.Firefox(options=firefox_option)
for course_code, course_name in zip(course_codes, course_names):
    url = f"https://studentvip.com.au/macquarie/subjects/{course_code}"
    driver.get(url)
    time.sleep(0.3)
    html = driver.page_source
    # Parse the HTML content using BeautifulSoup
    # Assuming you have the HTML content in the 'html' variable
    soup = BeautifulSoup(html, "html.parser")
    div_element = soup.find("div", class_="page-header")
    div_element2 = soup.find("h2", class_="subject-rating no-rating")
    if (div_element and "Oh no" in div_element.get_text()) or (
        div_element2 and "rate me" in div_element2.get_text()
    ):
        not_found.append(course_code)
        reviews.append("")
        stars.append("")
        continue
    div_element = soup.find("div", class_="rating")
    stars_count = len(div_element.find_all(class_="fas fa-star"))
    stars.append(stars_count)
    print(stars_count)

    div_element = soup.find_all("div", class_="list-group")
    div_element = div_element[-1]
    elements = div_element.find_all(["p"])
    elements2 = div_element.find_all(["small"])
    review = []
    for i in range(len(elements)):
        review.append(f"{elements[i].get_text()} - {elements2[i].get_text()}")
    reviews.append(review)

    print("Number of reviews:", len(reviews))
    print("Number of stars:", len(stars))
    print("Number of course codes:", len(course_codes))
    print("Number of course names:", len(course_names))


# Create a dictionary with the data
data_vip = {
    "reviews": reviews,
    "stars": stars,
    "Course Code": course_codes,
    "Course Name": course_names,
}

3
Number of reviews: 3
Number of stars: 3
Number of course codes: 35
Number of course names: 35
2
Number of reviews: 8
Number of stars: 8
Number of course codes: 35
Number of course names: 35
5
Number of reviews: 28
Number of stars: 28
Number of course codes: 35
Number of course names: 35
3
Number of reviews: 29
Number of stars: 29
Number of course codes: 35
Number of course names: 35
4
Number of reviews: 30
Number of stars: 30
Number of course codes: 35
Number of course names: 35
4
Number of reviews: 31
Number of stars: 31
Number of course codes: 35
Number of course names: 35
4
Number of reviews: 32
Number of stars: 32
Number of course codes: 35
Number of course names: 35
4
Number of reviews: 33
Number of stars: 33
Number of course codes: 35
Number of course names: 35


In [55]:
studentvip_df = pd.DataFrame(data_vip)

## Unit Task

In [56]:
not_found = []
all_tasks = []
all_date = []

unique_courses = df.drop_duplicates(subset="Course Code")
course_codes = unique_courses["Course Code"]
# course_codes = ["COMP1000"]
course_names = unique_courses["Course Name"]
firefox_option = Options()
firefox_option.add_argument("--headless")
driver = webdriver.Firefox(options=firefox_option)
for course_code, course_name in zip(course_codes, course_names):
    url = f"https://unitguides.mq.edu.au/units/archive_search?query={course_code}&year=2023"
    driver.get(url)
    html = driver.page_source
    # Parse the HTML content using BeautifulSoup
    # Assuming you have the HTML content in the 'html' variable
    soup = BeautifulSoup(html, "html.parser")

    # Find the <div> element with the class "alert alert-info"
    div_element = soup.find("div", class_="alert alert-info")

    # Check if the specific message is present in the HTML
    if div_element and "No results found" in div_element.get_text():
        # Update the URL or take any other desired action
        url = f"https://unitguides.mq.edu.au/units/archive_search?query={course_code}&year="
        driver.get(url)
        time.sleep(0.3)
        html = driver.page_source
        # Parse the HTML content using BeautifulSoup
        # Assuming you have the HTML content in the 'html' variable
        soup = BeautifulSoup(html, "html.parser")

        # Find the <div> element with the class "alert alert-info"
        div_element = soup.find("div", class_="alert alert-info")
        if div_element and "No results found" in div_element.get_text():
            not_found.append(course_code)
            all_tasks.append("")
            all_date.append("")
            continue

    try:
        # Find the div_offerings element by class name
        div_offerings = driver.find_element("css selector", ".table-search-results")

        try:
            print("Course Code:", course_code)
            print("Course Name:", course_name)
            # Find the first <a> tag within div_offerings
            a_tag = div_offerings.find_element("tag name", "a")

            # Click on the <a> tag
            a_tag.click()

            # Continue with the rest of your scraping logic or actions after the click
            # Find all <div> elements with class "general-info-value"

            # Extract the text within each <div> element
            html = driver.page_source
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(html, "html.parser")

            div_elements = soup.find("header", class_="unit-guide-header-container")
            h3 = div_elements.find("h3")
            print(h3.text.strip())
            all_date.append(h3.text.strip())

            div_elements = soup.find_all("tr", class_="assessment-task-row")
            task = []
            for div in div_elements:
                td_title = div.find("td", class_="title")
                td_weighting = div.find("td", class_="weighting")

                if td_title:
                    # Do something with the td_title
                    task.append(
                        f"{td_title.text.strip()} - {td_weighting.text.strip()}"
                    )
            print(task)
            all_tasks.append(task)

            print(len(all_date))
            print(len(all_tasks))
            print("------------------------------------")

        except NoSuchElementException:
            print("No <a> tag found within div_offerings")

    except NoSuchElementException:
        not_found.append(course_code)
        print("div_offerings element not found")


## EXPORT TO EXCEL

data_task = {
    "Course Code": course_codes,
    "Course Name": course_names,
    "Tasks": all_tasks,
    "Date": all_date,
}

Course Code: ABST1020
Course Name: Dharug Ngurra
2023 – Session 1, In person-scheduled-weekday, North Ryde
['Blogposts on themes - 30%', 'Participation - 30%', 'Reflective Essay - 40%']
1
1
------------------------------------
Course Code: ENVS1017
Course Name: The Living Environment
2023 – Session 1, In person-scheduled-infrequent, North Ryde
['Quizzes - 10%', 'Scientific Report - 30%', 'Environmental mapping and analysis - 30%', 'Final exam - 30%']
2
2
------------------------------------
Course Code: ENVS1018
Course Name: Environmental Management for a Changing World
2022 – Session 2, In person-scheduled-weekday, North Ryde
['Weekly tutorials - 0%', 'Online quiz - 15%', 'Data analysis - 30%', 'Podcast - 25%', 'Poster - 30%']
3
3
------------------------------------
Course Code: GEOP1010
Course Name: Introducing Human Geography
2023 – Session 1, In person-scheduled-weekday, North Ryde
['Geo Challenge - 30%', 'Research essay - 40%', 'Open-book task - 30%']
4
4
------------------------

In [57]:
unitguide_task_df = pd.DataFrame(data_task)

## Unit Guide

In [58]:
not_found = []
all_links = []
all_pre = []
all_copre = []
all_cobadged = []
all_des = []
all_final = []
all_presentation = []

unique_courses = df.drop_duplicates(subset="Course Code")
course_codes = unique_courses["Course Code"]
course_names = unique_courses["Course Name"]
firefox_option = Options()
firefox_option.add_argument("--headless")
driver = webdriver.Firefox(options=firefox_option)
for course_code, course_name in zip(course_codes, course_names):
    url = f"https://unitguides.mq.edu.au/units/archive_search?query={course_code}&year=2023"
    driver.get(url)
    time.sleep(0.3)
    html = driver.page_source
    # Parse the HTML content using BeautifulSoup
    # Assuming you have the HTML content in the 'html' variable
    soup = BeautifulSoup(html, "html.parser")

    # Find the <div> element with the class "alert alert-info"
    div_element = soup.find("div", class_="alert alert-info")

    # Check if the specific message is present in the HTML
    if div_element and "No results found" in div_element.get_text():
        # Update the URL or take any other desired action
        url = f"https://unitguides.mq.edu.au/units/archive_search?query={course_code}&year="
        driver.get(url)
        time.sleep(0.3)
        html = driver.page_source
        # Parse the HTML content using BeautifulSoup
        # Assuming you have the HTML content in the 'html' variable
        soup = BeautifulSoup(html, "html.parser")

        # Find the <div> element with the class "alert alert-info"
        div_element = soup.find("div", class_="alert alert-info")
        if div_element and "No results found" in div_element.get_text():
            not_found.append(course_code)
            all_links.append("")
            all_pre.append("")
            all_copre.append("")
            all_cobadged.append("")
            all_des.append("")
            all_final.append("No Idea")
            all_presentation.append("No Idea")
            continue

    try:
        # Find the div_offerings element by class name
        div_offerings = driver.find_element("css selector", ".table-search-results")

        try:
            print("Course Code:", course_code)
            print("Course Name:", course_name)
            # Find the first <a> tag within div_offerings
            a_tag = div_offerings.find_element("tag name", "a")

            # Click on the <a> tag
            a_tag.click()

            # Continue with the rest of your scraping logic or actions after the click
            # Find all <div> elements with class "general-info-value"

            # Extract the text within each <div> element
            html = driver.page_source
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(html, "html.parser")
            div_elements = soup.find_all("div", class_="general-info-value")
            words = [div.get_text(strip=True) for div in div_elements]
            print("Words:", len(words))

            # Print the extracted words
            current_url = driver.current_url

            # Print the current page URL
            print("Current URL:", current_url)
            all_links.append(current_url)
            all_pre.append(words[1])
            all_copre.append(words[2])
            all_cobadged.append(words[3])
            all_des.append(words[4])

            section = soup.find("section", id="assessment-tasks-section")

            # Extract the text within the <h3> element
            text = section.get_text()
            if "final exam" in text.lower():
                all_final.append("Possibly")
            else:
                all_final.append("No")

            if (
                "presentation" in text.lower()
                or "presentations" in text.lower()
                or "team" in text.lower()
            ):
                all_presentation.append("Possibly")
            else:
                all_presentation.append("No")
            print("-----------------------------------")

        except NoSuchElementException:
            print("No <a> tag found within div_offerings")

    except NoSuchElementException:
        not_found.append(course_code)
        print("div_offerings element not found")
    print(not_found)
    not_found_length = len(not_found)
    all_links_length = len(all_links)
    all_pre_length = len(all_pre)
    all_copre_length = len(all_copre)
    all_cobadged_length = len(all_cobadged)
    all_des_length = len(all_des)
    all_final_length = len(all_final)
    all_presentation_length = len(all_presentation)

    print("all_links length:", all_links_length)
    print("all_pre length:", all_pre_length)
    print("all_copre length:", all_copre_length)
    print("all_cobadged length:", all_cobadged_length)
    print("all_des length:", all_des_length)
    print("all_final length:", all_final_length)
    print("all_presentation length:", all_presentation_length)

## EXPORT TO EXCEL

data_guide = {
    "Course Code": course_codes,
    "Course Name": course_names,
    "Links": all_links,
    "Prerequisites": all_pre,
    "Corequisites": all_copre,
    "Co-badged Units": all_cobadged,
    "Description": all_des,
    "all_final": all_final,
    "all_presentation": all_presentation,
}

Course Code: ABST1020
Course Name: Dharug Ngurra
Words: 5
Current URL: https://unitguides.mq.edu.au/unit_offerings/155249/unit_guide
-----------------------------------
[]
all_links length: 1
all_pre length: 1
all_copre length: 1
all_cobadged length: 1
all_des length: 1
all_final length: 1
all_presentation length: 1
Course Code: ENVS1017
Course Name: The Living Environment
Words: 5
Current URL: https://unitguides.mq.edu.au/unit_offerings/157024/unit_guide
-----------------------------------
[]
all_links length: 2
all_pre length: 2
all_copre length: 2
all_cobadged length: 2
all_des length: 2
all_final length: 2
all_presentation length: 2
Course Code: ENVS1018
Course Name: Environmental Management for a Changing World
Words: 5
Current URL: https://unitguides.mq.edu.au/unit_offerings/149196/unit_guide
-----------------------------------
[]
all_links length: 3
all_pre length: 3
all_copre length: 3
all_cobadged length: 3
all_des length: 3
all_final length: 3
all_presentation length: 3
Cours

In [59]:
unitguide_df = pd.DataFrame(data_guide)

## Add major subjects

In [60]:
# Group units containing "elective" into "Elective units" category
df_degree.loc[df_degree['Category'].str.contains('elective', case=False), 'Category'] = 'Elective units'

# Group units containing "essential" into "Essential units" category
df_degree.loc[df_degree['Category'].str.contains('essential', case=False), 'Category'] = 'Essential units'

# Count the unique occurrences in the updated "Category" column
category_counts = df_degree['Category'].value_counts()

# Print the unique occurrences and their counts
print(category_counts)


Essential units    24
Elective units     12
Capstone unit       1
Name: Category, dtype: int64


In [61]:
filtered_data = df_degree[df_degree['Category'].isin(['Major', 'Specialisation', 'Capstone unit'])]

# Create a new DataFrame for the filtered rows
filtered_df = pd.DataFrame(filtered_data)
# Filter rows based on "credit point" in "Course Code" column
df_degree = df_degree[~df_degree['Course Code'].str.contains('credit point', case=False)]

# Reset the index after dropping rows
df_degree = df_degree.reset_index(drop=True)
# Group units containing "elective" into "Elective units" category
df_degree.loc[df_degree['Category'].str.contains('elective', case=False), 'Category'] = 'Elective units'

# Group units containing "essential" into "Essential units" category
df_degree.loc[df_degree['Category'].str.contains('essential', case=False), 'Category'] = 'Essential units'

# Separate majors and specializations into two lists
majors = list(df_degree[df_degree['Category'] == 'Major']['Course Code'])
specializations = list(df_degree[df_degree['Category'] == 'Specialisation']['Course Code'])
# Print the majors and specializations lists
print("Majors:")
print(majors)
print("\nSpecializations:")
print(specializations)
all_majors = majors + specializations

Majors:
[]

Specializations:
[]


In [62]:
for major in all_majors:
    ess = []
    elec = []
    url = f"https://coursehandbook.mq.edu.au/2023/aos/{major}"
    driver.get(url)
    time.sleep(0.3)
    html = driver.page_source
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    if driver.current_url == "https://coursehandbook.mq.edu.au/404-page-not-found":
        url = f"https://coursehandbook.mq.edu.au/2022/aos/{major}"
        driver.get(url)
        time.sleep(1)
        if driver.current_url == "https://coursehandbook.mq.edu.au/404-page-not-found":
            all_missing.append(major)
            all_ess.append("N/A")
            all_elec.append("N/A")
            all_title.append("N/A")
            continue
    
    # Major title
    div_academic_banner = soup.find('div', id='academic-item-banner').find("h2")
    # Print the text of the <h2> element
    title = div_academic_banner.text.strip()
    div_curriculum_structure = soup.find('div', id='curriculumStructure')
    div_accordion_container = div_curriculum_structure.find_all('div', {'name': 'ACCORDION_CONTAINER'})
    for div in div_accordion_container:
        essential_units = div.find('strong')
        div_subjects = div.find_all('div', {'filter': 'subject'})
        for div2 in div_subjects:
            div_grey_500 = div2.find_all('div', {'color': 'grey._500'})
            if "essential" in essential_units.text.lower():
                ess.append(div_grey_500[0].text)
            elif "elective" in essential_units.text.lower():
                elec.append(div_grey_500[0].text)
    print(title)
    print(ess)
    print(elec)
    print(all_missing)
    print("------------------")

    all_ess.append(ess)
    all_elec.append(elec)
    all_title.append(title)

In [63]:
# Create a DataFrame from the lists
data = pd.DataFrame({
    'Title': all_title,
    'Code': all_majors,
    'Essential Units': all_ess,
    'Elective Units': all_elec,
})


In [64]:
# Filter out rows with 'N/A' values
data = data.dropna()

# Create an empty list to store the transformed data
transformed_data = []

# Iterate over each row in the DataFrame
for _, row in data.iterrows():
    title = row['Title']
    code = row['Code']
    
    # Extract essential units and add them to the transformed data list
    ess_units = row['Essential Units']
    if ess_units:
        for ess_unit in eval(ess_units):
            transformed_data.append((title, code,'Essential units',  ess_unit))
    
    # Extract elective units and add them to the transformed data list
    elec_units = row['Elective Units']
    if elec_units:
        for elec_unit in eval(elec_units):
            transformed_data.append((title, code,'Elective units',  elec_unit))

# Create a DataFrame from the transformed data list
transformed_df = pd.DataFrame(transformed_data, columns=['Major Name', 'Major Code', 'Course Type', 'Course Code'])

# Print the transformed DataFrame

print(transformed_df)

Empty DataFrame
Columns: [Major Name, Major Code, Course Type, Course Code]
Index: []


In [65]:
# Remove duplicate course codes and get corresponding course names
unique_courses = transformed_df.drop_duplicates(subset="Course Code")
course_codes = unique_courses["Course Code"]
course_names = unique_courses["Major Name"]

all_course_code = []
all_course_name = []
period = []
error_list = []
# Scrape the website for each course code
for course_code, course_name in zip(course_codes, course_names):
    url = f"https://coursehandbook.mq.edu.au/2023/units/{course_code}"
    driver.get(url)
    time.sleep(0.3)
    html = driver.page_source
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # Course Name
    div_academic_banner = soup.find('div', id='academic-item-banner')
    if div_academic_banner:
        academic_banner_h2 = div_academic_banner.find("h2")
        # Print the text of the <h2> element
        title = academic_banner_h2.text.strip().split("-")[1].strip()
    else:
        title = ''

    # Find the div with the ID "Offerings"
    div_offerings = soup.find("div", id="Offerings")

    if div_offerings:
        # Find all div elements within the div_offerings element
        div_elements = div_offerings.find_all("div")
        print(len(div_elements))

        if len(div_elements) >= 2:
            # Get the second div element
            second_div = div_elements[3]
            sessions = second_div.get_text(strip=True).split("Session ")
            formatted_sessions = "\n".join(
                ["Session " + s for s in sessions if s.strip()]
            )
            print(formatted_sessions)
            period.append(formatted_sessions)
            all_course_code.append(course_code)
            all_course_name.append(title)

    # # Process the response as needed
    # # For example, you can print the course code, course name, and response status code
    print(f"Course Code: {course_code}")
    print(title)
    print("------------------------------")

## EXPORT TO EXCEL FILE
# Assuming you have the following lists:
# Create a DataFrame with the data
data = {
    "Course Code": all_course_code,
    "Course Name": all_course_name,
    "Period": period,
}
transformed_df = pd.DataFrame(data)


In [66]:
# Read the first Excel file into a DataFrame
df1 = pd.read_excel('course_name_major.xlsx')

# Read the second Excel file into a DataFrame
df2 = pd.read_excel('course_name_all.xlsx')

# Concatenate the two DataFrames vertically
concatenated_df = pd.concat([transformed_df, df2])
updated_df = concatenated_df.drop_duplicates()


In [67]:
# Export the DataFrame to an Excel file
filename = "course_name_all.xlsx"
sheet_name = "course_name_all"
concatenated_df.to_excel(filename, sheet_name=sheet_name, index=False)

## Degree of course

In [68]:
df2 = pd.read_excel("course_data3.xlsx")
df3 = pd.read_excel('major_unit.xlsx')

In [69]:
unit_degrees = {}

# Iterate over each unit in df1
for unit in df['Course Code']:
    unit_degrees[unit] = []

# Iterate over each row in df2
for index, row in df2.iterrows():
    unit = row['Course Code']
    degree = row['Degree']
    if unit in unit_degrees:
        unit_degrees[unit].append(degree)

# Print each unit with its corresponding degrees
for unit, degrees in unit_degrees.items():
    print(f"Unit: {unit}")
    print(f"Degrees: {degrees}")
    print()

Unit: ABST1020
Degrees: ['Bachelor of Planning']

Unit: ENVS1017
Degrees: ['Bachelor of Biodiversity and Conservation', 'Bachelor of Planning', 'Earth and Environmental Sciences']

Unit: ENVS1018
Degrees: ['Bachelor of Planning']

Unit: GEOP1010
Degrees: ['Bachelor of Planning']

Unit: GEOP1030
Degrees: ['Bachelor of Planning']

Unit: GEOP1040
Degrees: ['Bachelor of Planning']

Unit: GEOP1080
Degrees: ['Bachelor of Planning']

Unit: ENVS2364
Degrees: ['Bachelor of Biodiversity and Conservation', 'Bachelor of Planning', 'Earth and Environmental Sciences']

Unit: GEOP2010
Degrees: ['Bachelor of Planning']

Unit: GEOP2040
Degrees: ['Bachelor of Planning']

Unit: GEOP2050
Degrees: ['Bachelor of Planning', 'Bachelor of Social Science']

Unit: GEOP2060
Degrees: ['Bachelor of Planning']

Unit: GEOP2070
Degrees: ['Bachelor of Planning']

Unit: GEOP2080
Degrees: ['Bachelor of Planning']

Unit: GEOP3000
Degrees: ['Bachelor of Planning']

Unit: GEOP3020
Degrees: ['Bachelor of Planning']

Unit: GE

In [70]:
import ast
# Create a dictionary to store units and their corresponding degrees
unit_degrees2 = {}

# Iterate over each unit in df1
for unit in df['Course Code']:
    unit_degrees2[unit] = []

# Iterate over each row in df2
for index, row in df3.iterrows():
    major = row['Elective Units']
    major2 = row['Essential Units']
    title = row['Title']
    if isinstance(major, str):
        for m in ast.literal_eval(major):
            # print(m)
            if m in unit_degrees2:
                unit_degrees2[m].append(f"Major: {title}")
        for m in ast.literal_eval(major2):
            if m in unit_degrees2:
                unit_degrees2[m].append(f"Major: {title}")


unit_degrees2_filtered = {unit: degrees for unit, degrees in unit_degrees2.items() if degrees != []}

# Print each unit with its corresponding degrees
for unit, degrees in unit_degrees2.items():
    print(f"Unit: {unit}")
    print(f"Degrees: {degrees}")
    print()


Unit: ABST1020
Degrees: []

Unit: ENVS1017
Degrees: ['Major: Earth and Environmental Sciences']

Unit: ENVS1018
Degrees: []

Unit: GEOP1010
Degrees: []

Unit: GEOP1030
Degrees: []

Unit: GEOP1040
Degrees: []

Unit: GEOP1080
Degrees: []

Unit: ENVS2364
Degrees: ['Major: Earth and Environmental Sciences']

Unit: GEOP2010
Degrees: []

Unit: GEOP2040
Degrees: []

Unit: GEOP2050
Degrees: []

Unit: GEOP2060
Degrees: []

Unit: GEOP2070
Degrees: []

Unit: GEOP2080
Degrees: []

Unit: GEOP3000
Degrees: []

Unit: GEOP3020
Degrees: []

Unit: GEOP3030
Degrees: []

Unit: ABST3040
Degrees: []

Unit: GEOP3070
Degrees: []

Unit: GEOP3080
Degrees: []

Unit: GEOP3090
Degrees: []

Unit: GEOP4080
Degrees: []

Unit: GEOP4070
Degrees: []

Unit: LAWS5040
Degrees: []

Unit: GEOP4090
Degrees: []

Unit: COMP1050
Degrees: []

Unit: COMP1150
Degrees: []

Unit: COMP1300
Degrees: ['Major: Cyber Security Governance']

Unit: COMP1350
Degrees: ['Major: Business Information Systems']

Unit: COMP1750
Degrees: ['Major: In

In [71]:
merged_dict = unit_degrees.copy()

for key, value in unit_degrees2.items():
    if key in merged_dict:
        merged_dict[key].extend(value)
    else:
        merged_dict[key] = value

print(merged_dict)

{'ABST1020': ['Bachelor of Planning'], 'ENVS1017': ['Bachelor of Biodiversity and Conservation', 'Bachelor of Planning', 'Earth and Environmental Sciences', 'Major: Earth and Environmental Sciences'], 'ENVS1018': ['Bachelor of Planning'], 'GEOP1010': ['Bachelor of Planning'], 'GEOP1030': ['Bachelor of Planning'], 'GEOP1040': ['Bachelor of Planning'], 'GEOP1080': ['Bachelor of Planning'], 'ENVS2364': ['Bachelor of Biodiversity and Conservation', 'Bachelor of Planning', 'Earth and Environmental Sciences', 'Major: Earth and Environmental Sciences'], 'GEOP2010': ['Bachelor of Planning'], 'GEOP2040': ['Bachelor of Planning'], 'GEOP2050': ['Bachelor of Planning', 'Bachelor of Social Science'], 'GEOP2060': ['Bachelor of Planning'], 'GEOP2070': ['Bachelor of Planning'], 'GEOP2080': ['Bachelor of Planning'], 'GEOP3000': ['Bachelor of Planning'], 'GEOP3020': ['Bachelor of Planning'], 'GEOP3030': ['Bachelor of Planning'], 'ABST3040': ['Bachelor of Planning', 'Bachelor of Social Science'], 'GEOP30

In [72]:
degree_of_course_df = pd.DataFrame(merged_dict.items(), columns=['Course Code', 'Major'])

## Merge All

In [99]:
# Merge the dataframes on "Course Code" column
merged_df = pd.merge(degree_of_course_df, df, on='Course Code', how='outer')


# Merge "studentvip.xlsx" with the existing merged data
merged_df = pd.merge(merged_df, studentvip_df, on='Course Code', how='outer')


# Merge "unitguide.xlsx" with the existing merged data
merged_df = pd.merge(merged_df, unitguide_df, on='Course Code', how='outer')


# Merge "unitguide_task.xlsx" with the existing merged data
merged_df = pd.merge(merged_df, unitguide_task_df, on='Course Code', how='outer')

C:\Users\nplon\AppData\Local\Temp\ipykernel_85016\2611375818.py:14: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Course Name_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_df = pd.merge(merged_df, unitguide_task_df, on='Course Code', how='outer')


In [ ]:
# merged_df

In [100]:
merged_df = merged_df.rename(columns={'Course Name_x': 'Course Name', 'Links': 'Link',
                        'reviews': 'Reviews', 'stars': 'Star',
                        'all_final': 'Final Exam', 'all_presentation': 'Teamwork',
                        'Tasks': 'Assignment type', 'Date': 'Unit Date'})
# Drop the unwanted column
merged_df = merged_df.drop(columns='Course Name_y')
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

In [105]:
title_all = "final_merged_data"
# Existing DataFrame
existing_df = pd.read_excel(f"{title_all}.xlsx")
# Append the new data to the existing DataFrame
updated_df = existing_df.append(df_degree, ignore_index=True)
# Drop duplicates from the combined DataFrame
updated_df = updated_df.drop_duplicates()
# Export the DataFrame to an Excel file
filename = f"{title_all}.xlsx"
sheet_name = f"{title_all}"
updated_df.to_excel(filename, sheet_name=sheet_name, index=False)

C:\Users\nplon\AppData\Local\Temp\ipykernel_85016\2404109132.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updated_df = existing_df.append(df_degree, ignore_index=True)


In [104]:
column_types = updated_df.dtypes

print(column_types)

Course Code        object
Course Name        object
Major              object
Prerequisites      object
Period             object
Reviews            object
Star               object
Link               object
Corequisites       object
Co-badged Units    object
Description        object
Final Exam         object
Teamwork           object
Assignment type    object
Unit Date          object
dtype: object


In [93]:
# merged_df = merged_df.rename(columns={'Course Name_x': 'Course Name', 'Links': 'Link',
#                         'reviews': 'Reviews', 'stars': 'Star',
#                         'all_final': 'Final Exam', 'all_presentation': 'Teamwork',
#                         'Tasks': 'Assignment type', 'Date': 'Unit Date'})
# # Drop the unwanted column
# merged_df = merged_df.drop(columns='Course Name_y')
# merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]